# Demonstration of how to use SphericalHarmonicFeaturesVariationalStrategy in a shallow ApproximateGP

In [1]:
import geometric_kernels.torch


import torch 
import gpytorch 
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from geometric_kernels.spaces import Hypersphere
from linear_operator.operators import DiagLinearOperator
from mdgp.variational.spherical_harmonic_features.utils import * 
from mdgp.kernels import GeometricMaternKernel
from mdgp.utils import test_psd, sphere_uniform_grid, sphere_meshgrid, spherical_antiharmonic, spherical_harmonic
from tqdm.autonotebook import tqdm 


torch.set_default_dtype(torch.float64)
DEBUG: bool = True

INFO: Using numpy backend
/tmp/ipykernel_24199/3405421368.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
class gpytorchSGP(ApproximateGP):
    def __init__(self, max_ell, d, max_ell_prior, epsilon_sigma=1.0, kappa=1.0, nu=2.5, sigma=1.0, batch_shape=torch.Size([]), jitter_val=1e-6, optimize_nu: bool = True):
        m = total_num_harmonics(max_ell, d)
        variational_distribution = CholeskyVariationalDistribution(num_inducing_points=m, batch_shape=batch_shape)
        variational_strategy = SGPVariationalStrategy(self, variational_distribution, num_levels=max_ell, jitter_val=jitter_val)
        super().__init__(variational_strategy=variational_strategy)

        # constants 
        self.jitter_val = jitter_val
        self.max_ell = max_ell
        self.max_ell_prior = max_ell_prior
        self.d = d

        # modules 
        base_kernel = GeometricMaternKernel(
            space=Hypersphere(d),
            lengthscale=kappa, 
            nu=nu, 
            trainable_nu=optimize_nu, 
            num_eigenfunctions=max_ell_prior,
        )
        self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel)
        self.mean_module = gpytorch.means.ConstantMean()
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()

        # prior hyperparams 
        self.covar_module.outputscale = sigma ** 2
        self.likelihood.noise = epsilon_sigma ** 2

    def forward(self, x) -> MultivariateNormal:
        p_sigma = self.covar_module(x)
        p_mu = self.mean_module(x)
        return MultivariateNormal(p_mu, p_sigma)


class SGPVariationalStrategy(torch.nn.Module):
    def __init__(self, model: gpytorchSGP, variational_distribution: CholeskyVariationalDistribution, num_levels, jitter_val=1e-6):
        super().__init__()
        object.__setattr__(self, "_model", model)
        self._variational_distribution = variational_distribution
        self.jitter_val = jitter_val

        # Extract references to kernel parameters 
        self.num_levels = num_levels

    @property
    def model(self) -> gpytorchSGP:
        return self._model
    
    @property
    def variational_distribution(self) -> MultivariateNormal:
        return self._variational_distribution()
    
    @property
    def prior_distribution(self) -> MultivariateNormal:
        zeros = torch.zeros(
            self._variational_distribution.shape(),
            dtype=self._variational_distribution.dtype,
            device=self._variational_distribution.device,
        )
        ones = torch.ones_like(zeros)
        res = MultivariateNormal(zeros, DiagLinearOperator(ones))
        return res

    def forward(self, x) -> MultivariateNormal:
        # prior at x 
        px = self.model.forward(x)
        mu_x, Kxx = px.mean, px.lazy_covariance_matrix
        Kxx = Kxx.add_jitter(self.jitter_val)
        if DEBUG:
            test_psd(Kxx)

        # whitened prior at u
        Linv_pu = self.prior_distribution
        Linv_mu, Linv_Kuu_LTinv = Linv_pu.mean, Linv_pu.lazy_covariance_matrix

        # whitened variational posterior at u
        Linv_qu = self.variational_distribution
        Linv_m, Linv_S_LTinv = Linv_qu.mean, Linv_qu.lazy_covariance_matrix
        if DEBUG:
            test_psd(Linv_S_LTinv)

        # unwhitening + projection and vice-versa
        LT_Phiux = matern_LT_Phi_from_kernel(x, self.model.covar_module, num_levels=self.num_levels)
        Phixu_L = LT_Phiux.mT

        # posterior at x 
        qx_sigma = Kxx + Phixu_L @ (Linv_S_LTinv - Linv_Kuu_LTinv) @ LT_Phiux
        qx_sigma = qx_sigma.add_jitter(self.jitter_val)
        qx_mu = mu_x + Phixu_L @ (Linv_m - Linv_mu)

        if DEBUG:
            test_psd(qx_sigma)
        return MultivariateNormal(qx_mu, qx_sigma)
    
    def kl_divergence(self):
        with gpytorch.settings.max_preconditioner_size(0):
            kl_divergence = torch.distributions.kl.kl_divergence(self.variational_distribution, self.prior_distribution)
        return kl_divergence
    
    def __call__(self, x, prior: bool = False, **kwargs) -> MultivariateNormal:
        if prior:
            return self.model.forward(x)
        return super().__call__(x, **kwargs)

In [3]:
def get_data(num_train: int = 400, smooth: bool = True, noise_std: float = 1e-2, seed: int = 0) -> tuple[Tensor, Tensor, Tensor, Tensor]:
    torch.manual_seed(seed)

    f = spherical_harmonic if smooth else spherical_antiharmonic
    x_train = sphere_uniform_grid(num_train)
    f_train = f(x_train, 2, 3)
    y_train = f_train + noise_std * torch.randn_like(f_train)
    x_test = sphere_meshgrid(50, 50)
    f_test = f(x_test, 2, 3)
    y_test = f_test + noise_std * torch.randn_like(f_test)
    return x_train, y_train, x_test, y_test

In [4]:
smooth = True
d = 2
num_inducing_points = 100
max_ell = num_spherical_harmonics_to_num_levels(num_inducing_points, d)[0]
max_ell_prior = 30
num_epochs = 1000


model = gpytorchSGP(max_ell=max_ell, d=d, max_ell_prior=max_ell_prior, optimize_nu=True)
x_train, y_train, x_test, y_test = get_data(smooth=smooth)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, maximize=True)
elbo = gpytorch.mlls.VariationalELBO(model.likelihood, model, num_data=len(y_train))

model.train()
for i in (pbar := tqdm(range(num_epochs), desc="Training")):
    optimizer.zero_grad()
    output = model(x_train)
    loss = elbo(output, y_train)
    loss.backward()
    pbar.set_postfix(loss=loss.item())
    optimizer.step()


Training:   0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: matern_LT_Phi_from_kernel() missing 1 required positional argument: 'num_levels_prior'

In [24]:
from plotly import graph_objects as go
from plotly.subplots import make_subplots


def plot_predictions(model, x_test, y_test):
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Predicted", "True"), 
                        specs=[[{'type': 'surface'}, {'type': 'surface'}]])
    model.eval()
    with torch.no_grad():
        pred = model(x_test.view(-1, 3))
        pred_mean = pred.mean

    x, y, z = x_test.unbind(-1)
    predicted = go.Surface(x=x, y=y, z=z, surfacecolor=pred_mean)
    true = go.Surface(x=x, y=y, z=z, surfacecolor=y_test)
    fig.add_trace(predicted, row=1, col=1)
    fig.add_trace(true, row=1, col=2)
    fig.show()

In [29]:
plot_predictions(model, x_test, y_test)